## Simple example of ETL process on spotify songs we listened to

In [1]:
import sqlalchemy
import pandas as pd
import requests
import json
from datetime import datetime
import datetime

In [15]:
# Extracting through the spotify API
# Note: The token expires after about 10 minutes, so you have to take a new token every now and then.

DATABASE_LOCATION = 'sqlite:///my_played_tracks.sqlite'
USER_ID = 'pi_dzej'
TOKEN = "BQAD_iyRZtra_zrJqwQsjC6GAsEV-sIKQTm5xyOVdTkye_7n2OjHV6G8LYuReQYdAnQmZKiB0taDPVv726yhoxZJeOItW-DwwAwoYjmSKc0F_jqmTYzSiqu0QAXAJuT8PSWgM4tA9lAklmzQOl0FmoAjd0pLxboJTF1XeGub"
headers = {
     "Accept" : "application/json",
     "Content-Type" : "application/json",
     "Authorization" : f"Bearer {TOKEN}"
}

In [27]:
# defining the time period in which we want to know what song we listened to
yesterday = today - datetime.timedelta(days = 15)

datetime.datetime(2022, 1, 24, 13, 8, 49, 68295)

In [28]:
# convert it into a unix miliseconds timestamp
yesterday_unix_timestamp = int(yesterday.timestamp()) * 1000

1643026129000

In [29]:
# getting the data from the spotify API
r = requests.get(f"https://api.spotify.com/v1/me/player/recently-played?after={yesterday_unix_timestamp}", headers = headers)

In [30]:
data = r.json()

In [25]:
song_names = []
artist_names = []
played_at_list = []
timestamps = []

In [54]:
for song in data['items']:
    song_names.append(song["track"]["name"])
    artist_names.append(song["track"]["artists"][0]["name"])
    played_at_list.append(song['played_at'])
    timestamps.append(song['played_at'][0:10])    

In [65]:
song_df = pd.DataFrame(data = [song_names,artist_names,played_at_list,timestamps])

In [68]:
song_df = song_df.transpose()

In [73]:
song_df.columns = ['song_name','artist_name','played_at','date_played']

In [75]:
print(song_df)

                   song_name        artist_name                 played_at  \
0                       Kaze       Tophat Panda  2022-02-04T21:46:57.928Z   
1                     Lights  Tibeauthetraveler  2022-02-04T21:44:36.691Z   
2                    Chanson     lechiffrebeats  2022-02-04T21:42:23.923Z   
3              Sunset Cliffs                CMJ  2022-02-04T21:39:34.735Z   
4               Scary Coffee           JazzyHan  2022-02-04T21:37:21.853Z   
5                Spring Rain                BVG  2022-02-04T21:35:23.372Z   
6                    Stories     Rebecka Aether  2022-02-04T18:06:21.475Z   
7                Piece of me        Orchid Rain  2022-02-04T16:04:47.290Z   
8   When It Was Just A Dream         Diya Wiler  2022-02-04T16:01:56.178Z   
9                      Sajda        Ijaya Gupta  2022-02-04T15:58:42.607Z   
10                      Aura         Zion Llama  2022-02-04T15:55:14.142Z   
11               Relaksohuni            Qetesia  2022-02-04T15:52:55.135Z   

In [101]:
# Transform
def data_validation_check(df):
   # checking if data is empty
    if df.empty:
        print("No songs played in the last 15 days.")
        return False
    # we can only listen to one song at a time, so 'played_at' is our primary key
    if df['played_at'].is_unique:
        pass
    else:
        raise Exception("Primary Key Check Violated.")
    # check for null values in our data
    if df.isnull().values.any():
        raise Exception("Null values found.")
    
    return True

In [104]:
if data_validation_check(song_df):
    print('Data valid, proceed to Load stage')

Data valid, proceed to Load stage


In [105]:
# Load
engine = sqlalchemy.create_engine(DATABASE_LOCATION)

In [106]:
try:
    song_df.to_sql("my_played_tracks", engine, index = False, if_exists = 'append')
except:
    print('Data already exists')